In [ ]:
from pymongo import MongoClient

# What is a Cursor?

# A cursor in MongoDB (and in databases generally) is a mechanism that allows you to iterate over a set of documents or records returned by a query, one document at a time. Instead of loading all documents in memory at once, a cursor helps manage large datasets by fetching documents as you process them.

# ------------------------------------------------------------------------------------------
# Purpose of a Cursor

# Efficient Data Handling: Instead of loading all records into memory, a cursor lets you fetch records incrementally, saving memory and resources.
# Controlled Iteration: Cursors allow you to process each document one by one, making it possible to apply processing logic or transformations to each document before moving to the next.
# Better Performance: Cursors improve performance when working with large data by avoiding bulk fetching of records.

# ------------------------------------------------------------------------------------------

# When to Use a Cursor

# When processing a large number of records from a database.
# When you need to apply some logic to each document individually.
# When you want to optimize memory usage by loading documents only as needed.

# ------------------------------------------------------------------------------------------

# Explanation of Cursor Usage

# departments_cursor = departments_collection.find(): This creates a cursor to iterate through each department document.
# for department in departments_cursor:: Here, we use the cursor to access each department one by one.
# employees_cursor = employees_collection.find({'department_id': department['_id']}): For each department, we create a new cursor to retrieve associated employees based on the department_id.
# for employee in employees_cursor:: We use a nested cursor to iterate through each employee belonging to the current department.

# ------------------------------------------------------------------------------------------

# Why Use a Cursor?

# Efficient fetching of records without loading everything at once.
# Controlled access to each document, letting us process departments and employees independently.
# Reduced memory usage, as only the required documents are fetched when needed.

# ------------------------------------------------------------------------------------------

# Use Cases for Cursors

# Large Dataset Processing: When working with large collections, cursors prevent memory overload by fetching documents only when needed.
# Data Transformation: Cursors allow for real-time data processing, enabling transformations on each document during iteration.
# Pagination: Cursors are often used to implement pagination by limiting the number of documents fetched at a time.


# Connect to MongoDB (update the connection string if necessary)
client = MongoClient("mongodb://localhost:27017/")
db = client['MCA_WT']

# Define collections
departments_collection = db['Department']
employees_collection = db['Employee']

# Insert sample records (if collections are empty, to avoid duplicates)
if departments_collection.count_documents({}) == 0:
    departments = [
        {"_id": 1, "name": "Engineering"},
        {"_id": 2, "name": "Human Resources"},
        {"_id": 3, "name": "Sales"}
    ]
departments_collection.insert_many(departments)
print("Inserted departments.")

if employees_collection.count_documents({}) == 0:
    employees = [
        {"_id": 101, "name": "Alice", "department_id": 1, "position": "Software Engineer"},
        {"_id": 102, "name": "Bob", "department_id": 1, "position": "Data Scientist"},
        {"_id": 103, "name": "Charlie", "department_id": 2, "position": "HR Manager"},
        {"_id": 104, "name": "David", "department_id": 3, "position": "Sales Executive"}
    ]
employees_collection.insert_many(employees)
print("Inserted employees.")

# Fetch department-wise employee information using cursors
departments_cursor = departments_collection.find()  # Cursor for departments

for department in departments_cursor:  # Iterate using the cursor
    print(f"\nDepartment: {department['name']}")
    
    # Find employees associated with this department using a cursor
    employees_cursor = employees_collection.find({'department_id': department['_id']})
    
    # Check if the department has employees
    if employees_cursor:
        for employee in employees_cursor:  # Cursor for employees
            print(f"  Employee Name: {employee['name']}, Position: {employee['position']}")
    else:
        print("  No employees found.")
# Close the connection
client.close()

# ---------------------------------------------------------------------------------

# departments = [
#         {"_id": 1, "name": "Engineering", 'employees':[101,102]},
#         {"_id": 2, "name": "Human Resources",'employees':[103]},
#         {"_id": 3, "name": "Sales",'employees':[104]}
#     ]



# var cursor = db.Department.find();


## we have departments :- Eng, hr, sales

# while (cursor.hasNext()) {
#     var department = cursor.next();

#     print("Department: " + department.name);

#     var employees = db.Employee.find({ "_id" : {"$in" : department.employees} });

#     while (employees.hasNext()) {
#         printjson(employees.next());
#     }
#     print('\n');
# }

# hasNext(): This method checks if there are more documents to be fetched from the cursor. It returns true if there are additional documents in the result set, otherwise false.
# next(): This method retrieves the next document in the cursor and moves the cursor forward to the subsequent document.



Department: Engineering
  Employee Name: Alice, Position: Software Engineer
  Employee Name: Bob, Position: Data Scientist

Department: Human Resources
  Employee Name: Charlie, Position: HR Manager

Department: Sales
  Employee Name: David, Position: Sales Executive


In [ ]:
# 1. $unwind
# The $unwind stage is used to "deconstruct" an array field, meaning it creates separate documents for each item in the array.

{
  "_id": 1,
  "name": "John Doe",
  "movies": ["movie1", "movie2", "movie3"]
}

{"$unwind": "$movies"}

# o/p

{ "_id": 1, "name": "John Doe", "movies": "movie1" }
{ "_id": 1, "name": "John Doe", "movies": "movie2" }
{ "_id": 1, "name": "John Doe", "movies": "movie3" }


# # --------------------------------------------------------------------------------------------

# 2. $lookup
# The $lookup stage performs a "left outer join" between collections, allowing us to combine data from two collections.

# Producers:
{ "_id": 1, "name": "John Doe", "movies": ["movie1", "movie2"] }


# Movies:
{ "_id": "movie1", "title": "Movie One", "year": 2023 }
{ "_id": "movie2", "title": "Movie Two", "year": 2024 }



# To combine these collections:

{
  "$lookup": {
    "from": "Movies",
    "localField": "movies",
    "foreignField": "_id",
    "as": "movie_details"
  }
}


# from: Specifies the target collection (Movies).
# localField: Specifies the field in the current collection (movies) to match.
# foreignField: Specifies the field in the Movies collection (_id) to match against.
# as: Defines a new field where matched documents will be stored (movie_details).


# o/p 

{
  "_id": 1,
  "name": "John Doe",
  "movies": ["movie1", "movie2"],
  "movie_details": [
    { "_id": "movie1", "title": "Movie One", "year": 2023 },
    { "_id": "movie2", "title": "Movie Two", "year": 2024 }
  ]
}

# # --------------------------------------------------------------------------------------------


# 3. $match
# The $match stage filters documents in a pipeline, keeping only those that satisfy specified conditions.



# Using the previous Producers documents, if we want only those with a name of "John Doe":
{"$match": {"name": "John Doe"}}

# o/p 

{ "_id": 1, "name": "John Doe", "movies": ["movie1", "movie2"] }


# # --------------------------------------------------------------------------------------------

# 4. $project

# The $project stage specifies which fields to include or exclude in the output. It’s used to shape the final output.

{ "_id": 1, "name": "John Doe", "movies": ["movie1", "movie2"], "age": 45 }


# If we only want name and movies in the output:

{"$project": {"_id": 0, "name": 1, "movies": 1}}


# _id: 0: Excludes the _id field from the output.
# name: 1 and movies: 1: Includes the name and movies fields in the output.

# Result:
{ "name": "John Doe", "movies": ["movie1", "movie2"] }


# next

[
  {"_id": 1, "title": "Inception", "director": "Christopher Nolan", "year": 2010, "genre": "Sci-Fi", "rating": 8.8, "duration_minutes": 148},
  {"_id": 2, "title": "Titanic", "director": "James Cameron", "year": 1997, "genre": "Romance", "rating": 7.8, "duration_minutes": 195},
  {"_id": 3, "title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "genre": "Action", "rating": 9.0, "duration_minutes": 152},
  {"_id": 4, "title": "The Godfather", "director": "Francis Ford Coppola", "year": 1972, "genre": "Crime", "rating": 9.2, "duration_minutes": 175},
  {"_id": 5, "title": "Avengers: Endgame", "director": "Anthony and Joe Russo", "year": 2019, "genre": "Action", "rating": 8.4, "duration_minutes": 181}
]

# Example 1: Exclude _id and Include Specific Fields
# You want to exclude the _id field and include the title, director, and rating.

db.movies.aggregate([
  {
    "$project": {
      "_id": 0,           # Exclude _id
      "title": 1,         # Include title
      "director": 1,      # Include director
      "rating": 1         # Include rating
    }
  }
])

# "_id": 0: Excludes the _id field.
# "title": 1, "director": 1, "rating": 1: Includes these fields in the output.

# o/p

[
  {"title": "Inception", "director": "Christopher Nolan", "rating": 8.8},
  {"title": "Titanic", "director": "James Cameron", "rating": 7.8},
  {"title": "The Dark Knight", "director": "Christopher Nolan", "rating": 9.0},
  {"title": "The Godfather", "director": "Francis Ford Coppola", "rating": 9.2},
  {"title": "Avengers: Endgame", "director": "Anthony and Joe Russo", "rating": 8.4}
]


# Renaming Fields and Adding Computed Field
# You want to rename title to movie_name, and calculate a new field movie_duration_hours by converting the duration_minutes to hours (i.e., dividing by 60).

db.movies.aggregate([
  {
    "$project": {
      "_id": 0,                           # Exclude _id
      "movie_name": "$title",             # Rename title to movie_name ( means we are changing field name from title to movie_name)
      "director": 1,                      # Include director
      "rating": 1,                        # Include rating
      "movie_duration_hours": {
        "$divide": ["$duration_minutes", 60]  # Create a new field for movie duration in hours
      }
    }
  }
])

# "movie_name": "$title": Renames the title field to movie_name.
# "movie_duration_hours": { "$divide": ["$duration_minutes", 60] }: Adds a new field movie_duration_hours, which is calculated by dividing duration_minutes by 60.

# o/p

[
  {"movie_name": "Inception", "director": "Christopher Nolan", "rating": 8.8, "movie_duration_hours": 2.47},
  {"movie_name": "Titanic", "director": "James Cameron", "rating": 7.8, "movie_duration_hours": 3.25},
  {"movie_name": "The Dark Knight", "director": "Christopher Nolan", "rating": 9.0, "movie_duration_hours": 2.53},
  {"movie_name": "The Godfather", "director": "Francis Ford Coppola", "rating": 9.2, "movie_duration_hours": 2.92},
  {"movie_name": "Avengers: Endgame", "director": "Anthony and Joe Russo", "rating": 8.4, "movie_duration_hours": 3.02}
]




# # --------------------------------------------------------------------------------------------


# 5. from, as, localField, and foreignField in $lookup
# These fields are all part of the $lookup stage.


{
  "$lookup": {
    "from": "Movies",
    "localField": "movies",
    "foreignField": "_id",
    "as": "movie_details"
  }
}


# from: The name of the collection to join, Movies in this case.
# as: Defines the new field in the output where matched documents are stored, here movie_details.
# localField: The field in the current collection (movies in Producers) used for matching.
# foreignField: The field in the Movies collection (_id) that must match the localField.

# # ----------------------------------------------------------------------

# $elemMatch
# To match array elements based on specific conditions.
# Usage: Often used in queries or within the $match stage of an aggregation pipeline.

{
  "_id": 1,
  "title": "Movie One",
  "actors": [
    { "name": "John", "role": "Hero", "age": 35 },
    { "name": "Alice", "role": "Villain", "age": 30 }
  ]
},
{
  "_id": 2,
  "title": "Movie Two",
  "actors": [
    { "name": "Akshay", "role": "Hero", "age": 45 },
    { "name": "John", "role": "Sidekick", "age": 35 }
  ]
}


# Query

db.Movies.find({
  "actors": {
    "$elemMatch": {
      "name": "John",
      "role": "Hero"
    }
  }
})


# "actors": The array field containing actor objects.
# $elemMatch: This operator allows you to specify a condition on elements inside the actors array. It finds documents where at least one element( record ) matches the specified criteria.
# Condition: The element inside the actors array must have the name equal to "John" and the role equal to "Hero".


# o/p 

{
  "_id": 1,
  "title": "Movie One",
  "actors": [
    { "name": "John", "role": "Hero", "age": 35 },
    { "name": "Alice", "role": "Villain", "age": 30 }
  ]
}

# ------------------------------------------------------------------------
# $group
# To aggregate data by grouping documents based on one or more fields.
# Usage: Commonly used with aggregation functions such as $sum, $avg, $min, $max, $push, and $addToSet.

# _id in $group: Defines the grouping key. Documents that share the same _id value are grouped together.

# Suppose you have a Movies collection that looks like this:

{ "_id": 1, "title": "Movie A", "genre": "Action", "release_year": 2021, "box_office": 500000 },
{ "_id": 2, "title": "Movie B", "genre": "Action", "release_year": 2021, "box_office": 300000 },
{ "_id": 3, "title": "Movie C", "genre": "Drama", "release_year": 2022, "box_office": 400000 },
{ "_id": 4, "title": "Movie D", "genre": "Drama", "release_year": 2021, "box_office": 200000}


# To calculate the total box office revenue for each genre, you can use $group like this:

db.Movies.aggregate([
    {
        "$group": {
            "_id": "$genre",
            "total_box_office": { "$sum": "$box_office" }
        }
    }
])
# _id in $group
# In $group, the _id field is used to define how the documents are grouped. This can be a single field, multiple fields, or even a computed value.
# _id: "$genre": The documents will be grouped by the genre field. Each unique genre value will become a _id in the result.
# total_box_office: { "$sum": "$box_office" }: Calculates the total box office for each genre by summing up the box_office values in each group.



# o/p 

{ "_id": "Action", "total_box_office": 800000 },
{ "_id": "Drama", "total_box_office": 600000 }



# next

[
  {"movie_name": "Inception", "director": "Christopher Nolan", "rating": 8.8, "movie_duration_hours": 2.47},
  {"movie_name": "Titanic", "director": "James Cameron", "rating": 7.8, "movie_duration_hours": 3.25},
  {"movie_name": "The Dark Knight", "director": "Christopher Nolan", "rating": 9.0, "movie_duration_hours": 2.53},
  {"movie_name": "The Godfather", "director": "Francis Ford Coppola", "rating": 9.2, "movie_duration_hours": 2.92},
  {"movie_name": "Avengers: Endgame", "director": "Anthony and Joe Russo", "rating": 8.4, "movie_duration_hours": 3.02}
]


{
  "$group": {
    "_id": { 
      "producer_id": "$_id", 
      "name": "$name", 
      "year": "$movie_details.release_year"
    },
    "movie_count": { "$sum": 1 }
  }
}


# First, the documents are grouped by the combination of producer_id, name, and year:
# Movies with the same producer, name, and release year will be grouped together.


# Group 1:

# producer_id: P1, name: "Movie A", year: 2021
# This group includes 3 movies.
# movie_count: 3
# Group 2:

# producer_id: P2, name: "Movie A", year: 2021
# This group includes 1 movie.
# movie_count: 1
# Group 3:

# producer_id: P1, name: "Movie A", year: 2022
# This group includes 1 movie.
# movie_count: 1


[
  {
    "_id": { "producer_id": "P1", "name": "Movie A", "year": 2021 },
    "movie_count": 3
  },
  {
    "_id": { "producer_id": "P2", "name": "Movie A", "year": 2021 },
    "movie_count": 1
  },
  {
    "_id": { "producer_id": "P1", "name": "Movie A", "year": 2022 },
    "movie_count": 1
  }
]


In [24]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['MCA_WT']

# Define collections
actors_collection = db['Actors']
producers_collection = db['Producers']
movies_collection = db['Movies']

actors_collection.delete_many({})
producers_collection.delete_many({})
movies_collection.delete_many({})
# Insert sample actors
actors = [
    {"_id": 1, "name": "Akshay", "movies": [1, 2]},
    {"_id": 2, "name": "John", "movies": [2, 3]},
    {"_id": 3, "name": "Amit", "movies": [1]},
    {"_id": 4, "name": "Priyanka", "movies": [3, 4]},
    {"_id": 5, "name": "Deepika", "movies": [4, 5]}
]
actors_collection.insert_many(actors)

# Insert sample producers
producers = [
    {"_id": 1, "name": "Karan", "movies": [1, 3]},
    {"_id": 2, "name": "Yash", "movies": [2]},
    {"_id": 3, "name": "Ekta", "movies": [3, 4]},
    {"_id": 4, "name": "Aditya", "movies": [4, 5]},
    {"_id": 5, "name": "Rohit", "movies": [2, 5]}
]
producers_collection.insert_many(producers)

# Insert sample movies
movies = [
    {"_id": 1, "title": "Movie A", "release_year": 2021, "actors": [{"actor_id": 1, "role": "Lead"}, {"actor_id": 3, "role": "Supporting"}], "producers": [1]},
    {"_id": 2, "title": "Movie B", "release_year": 2021, "actors": [{"actor_id": 1, "role": "Lead"}, {"actor_id": 2, "role": "Supporting"}], "producers": [2, 5]},
    {"_id": 3, "title": "Movie C", "release_year": 2022, "actors": [{"actor_id": 2, "role": "Lead"}, {"actor_id": 4, "role": "Supporting"}], "producers": [1, 3]},
    {"_id": 4, "title": "Movie D", "release_year": 2022, "actors": [{"actor_id": 4, "role": "Lead"}, {"actor_id": 5, "role": "Supporting"}], "producers": [3, 4]},
    {"_id": 5, "title": "Movie E", "release_year": 2022, "actors": [{"actor_id": 5, "role": "Lead"}], "producers": [4, 5]}
]
movies_collection.insert_many(movies)

# Query 1: Display producers who have produced more than one movie in a year
pipeline = [
    {"$unwind": "$movies"},
    {"$lookup": {
        "from": "Movies",
        "localField": "movies",
        "foreignField": "_id",
        "as": "movie_details"
    }},
    {"$unwind": "$movie_details"},
    {"$group": {
        "_id": {"producer_id": "$_id", "name": "$name", "year": "$movie_details.release_year"},
        "movie_count": {"$sum": 1}
    }},
    {"$match": {"movie_count": {"$gt": 1}}},
    {"$project": {"_id": 0, "producer_id": "$_id.producer_id", "name": "$_id.name", "year": "$_id.year", "movie_count": 1}}
]

producers_with_multiple_movies = list(producers_collection.aggregate(pipeline))
print("Producers who have produced more than one movie in a year:")
for producer in producers_with_multiple_movies:
    print(producer)

# Query 2: List names of actors who have acted in at least one movie in which 'Akshay' has acted
# Step 1: Find movies where Akshay acted
akshay_movies = actors_collection.find_one({"name": "Akshay"})["movies"]

# Step 2: Find actors in the same movies
other_actors = actors_collection.find({"movies": {"$in": akshay_movies}, "name": {"$ne": "Akshay"}}) #$ne operator in MongoDB stands for "not equal to"

print("\nActors who have acted in at least one movie with 'Akshay':")
for actor in other_actors:
    print(actor["name"])

# Close the connection


Producers who have produced more than one movie in a year:
{'movie_count': 2, 'producer_id': 4, 'name': 'Aditya', 'year': 2022}
{'movie_count': 2, 'producer_id': 3, 'name': 'Ekta', 'year': 2022}

Actors who have acted in at least one movie with 'Akshay':
John
Amit


# Step by step solving 

In [25]:
# [
#     {"_id": 1, "name": "Karan", "movies": [1, 3]},
#     {"_id": 2, "name": "Yash", "movies": [2]},
#     {"_id": 3, "name": "Ekta", "movies": [3, 4]},
#     {"_id": 4, "name": "Aditya", "movies": [4, 5]},
#     {"_id": 5, "name": "Rohit", "movies": [2, 5]}
# ]
pipeline = [
    {"$unwind": "$movies"},]
producers_with_multiple_movies = list(producers_collection.aggregate(pipeline))
for producer in producers_with_multiple_movies:
    print(producer)



{'_id': 1, 'name': 'Karan', 'movies': 1}
{'_id': 1, 'name': 'Karan', 'movies': 3}
{'_id': 2, 'name': 'Yash', 'movies': 2}
{'_id': 3, 'name': 'Ekta', 'movies': 3}
{'_id': 3, 'name': 'Ekta', 'movies': 4}
{'_id': 4, 'name': 'Aditya', 'movies': 4}
{'_id': 4, 'name': 'Aditya', 'movies': 5}
{'_id': 5, 'name': 'Rohit', 'movies': 2}
{'_id': 5, 'name': 'Rohit', 'movies': 5}


In [32]:
pipeline = [
    {"$unwind": "$movies"},
        # {'_id': 1, 'name': 'Karan', 'movies': 1}
        # {'_id': 1, 'name': 'Karan', 'movies': 3}
        # {'_id': 2, 'name': 'Yash', 'movies': 2}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5}

    
    {"$lookup": {
        "from": "Movies",
        "localField": "movies",
        "foreignField": "_id",
        "as": "movie_details"
    }},
    {"$unwind": "$movie_details"},
    
   
]

producers_with_multiple_movies = list(producers_collection.aggregate(pipeline))
for producer in producers_with_multiple_movies:
    print(producer)

{'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': {'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}}
{'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
{'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
{'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
{'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 

In [ ]:
pipeline = [
    {"$unwind": "$movies"},
        # {'_id': 1, 'name': 'Karan', 'movies': 1}
        # {'_id': 1, 'name': 'Karan', 'movies': 3}
        # {'_id': 2, 'name': 'Yash', 'movies': 2}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5}

    
    {"$lookup": {
        "from": "Movies",
        "localField": "movies",
        "foreignField": "_id",
        "as": "movie_details"
    }},

        # {'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': [{'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}]}
        # {'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': [{'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}]}
        # {'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': [{'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}]}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': [{'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}]}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': [{'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}]}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4, 'movie_details': [{'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}]}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5, 'movie_details': [{'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}]}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2, 'movie_details': [{'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}]}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5, 'movie_details': [{'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}]}
    
    {"$unwind": "$movie_details"},
    
        # {'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': {'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}}
        # {'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
        # {'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5, 'movie_details': {'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5, 'movie_details': {'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}}
    
    {"$group": {
        "_id": {"producer_id": "$_id", "name": "$name", "year": "$movie_details.release_year"},
        "movie_count": {"$sum": 1}
    }},
    
    
]

producers_with_multiple_movies = list(producers_collection.aggregate(pipeline))
for producer in producers_with_multiple_movies:
    print(producer)

{'_id': {'producer_id': 1, 'name': 'Karan', 'year': 2021}, 'movie_count': 1}
{'_id': {'producer_id': 3, 'name': 'Ekta', 'year': 2022}, 'movie_count': 2}
{'_id': {'producer_id': 4, 'name': 'Aditya', 'year': 2022}, 'movie_count': 2}
{'_id': {'producer_id': 5, 'name': 'Rohit', 'year': 2021}, 'movie_count': 1}
{'_id': {'producer_id': 5, 'name': 'Rohit', 'year': 2022}, 'movie_count': 1}
{'_id': {'producer_id': 2, 'name': 'Yash', 'year': 2021}, 'movie_count': 1}
{'_id': {'producer_id': 1, 'name': 'Karan', 'year': 2022}, 'movie_count': 1}


In [36]:
pipeline = [
    {"$unwind": "$movies"},
        # {'_id': 1, 'name': 'Karan', 'movies': 1}
        # {'_id': 1, 'name': 'Karan', 'movies': 3}
        # {'_id': 2, 'name': 'Yash', 'movies': 2}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5}

    
    {"$lookup": {
        "from": "Movies",
        "localField": "movies",
        "foreignField": "_id",
        "as": "movie_details"
    }},

        # {'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': [{'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}]}
        # {'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': [{'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}]}
        # {'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': [{'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}]}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': [{'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}]}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': [{'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}]}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4, 'movie_details': [{'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}]}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5, 'movie_details': [{'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}]}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2, 'movie_details': [{'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}]}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5, 'movie_details': [{'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}]}
    
    {"$unwind": "$movie_details"},
    
        # {'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': {'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}}
        # {'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
        # {'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5, 'movie_details': {'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5, 'movie_details': {'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}}
    
    {"$group": {
        "_id": {"producer_id": "$_id", "name": "$name", "year": "$movie_details.release_year"},
        "movie_count": {"$sum": 1}
    }},

        # {'_id': {'producer_id': 1, 'name': 'Karan', 'year': 2021}, 'movie_count': 1}
        # {'_id': {'producer_id': 3, 'name': 'Ekta', 'year': 2022}, 'movie_count': 2}
        # {'_id': {'producer_id': 4, 'name': 'Aditya', 'year': 2022}, 'movie_count': 2}
        # {'_id': {'producer_id': 5, 'name': 'Rohit', 'year': 2021}, 'movie_count': 1}
        # {'_id': {'producer_id': 5, 'name': 'Rohit', 'year': 2022}, 'movie_count': 1}
        # {'_id': {'producer_id': 2, 'name': 'Yash', 'year': 2021}, 'movie_count': 1}
        # {'_id': {'producer_id': 1, 'name': 'Karan', 'year': 2022}, 'movie_count': 1}


    {"$match": {"movie_count": {"$gt": 1}}},
    
]

producers_with_multiple_movies = list(producers_collection.aggregate(pipeline))
for producer in producers_with_multiple_movies:
    print(producer)

{'_id': {'producer_id': 3, 'name': 'Ekta', 'year': 2022}, 'movie_count': 2}
{'_id': {'producer_id': 4, 'name': 'Aditya', 'year': 2022}, 'movie_count': 2}


In [38]:
pipeline = [
    {"$unwind": "$movies"},
        # {'_id': 1, 'name': 'Karan', 'movies': 1}
        # {'_id': 1, 'name': 'Karan', 'movies': 3}
        # {'_id': 2, 'name': 'Yash', 'movies': 2}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5}

    
    {"$lookup": {
        "from": "Movies",
        "localField": "movies",
        "foreignField": "_id",
        "as": "movie_details"
    }},

        # {'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': [{'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}]}
        # {'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': [{'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}]}
        # {'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': [{'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}]}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': [{'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}]}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': [{'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}]}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4, 'movie_details': [{'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}]}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5, 'movie_details': [{'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}]}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2, 'movie_details': [{'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}]}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5, 'movie_details': [{'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}]}
    
    {"$unwind": "$movie_details"},
    
        # {'_id': 1, 'name': 'Karan', 'movies': 1, 'movie_details': {'_id': 1, 'title': 'Movie A', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 3, 'role': 'Supporting'}], 'producers': [1]}}
        # {'_id': 1, 'name': 'Karan', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
        # {'_id': 2, 'name': 'Yash', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
        # {'_id': 3, 'name': 'Ekta', 'movies': 3, 'movie_details': {'_id': 3, 'title': 'Movie C', 'release_year': 2022, 'actors': [{'actor_id': 2, 'role': 'Lead'}, {'actor_id': 4, 'role': 'Supporting'}], 'producers': [1, 3]}}
        # {'_id': 3, 'name': 'Ekta', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}}
        # {'_id': 4, 'name': 'Aditya', 'movies': 4, 'movie_details': {'_id': 4, 'title': 'Movie D', 'release_year': 2022, 'actors': [{'actor_id': 4, 'role': 'Lead'}, {'actor_id': 5, 'role': 'Supporting'}], 'producers': [3, 4]}}
        # {'_id': 4, 'name': 'Aditya', 'movies': 5, 'movie_details': {'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}}
        # {'_id': 5, 'name': 'Rohit', 'movies': 2, 'movie_details': {'_id': 2, 'title': 'Movie B', 'release_year': 2021, 'actors': [{'actor_id': 1, 'role': 'Lead'}, {'actor_id': 2, 'role': 'Supporting'}], 'producers': [2, 5]}}
        # {'_id': 5, 'name': 'Rohit', 'movies': 5, 'movie_details': {'_id': 5, 'title': 'Movie E', 'release_year': 2022, 'actors': [{'actor_id': 5, 'role': 'Lead'}], 'producers': [4, 5]}}
    
    {"$group": {
        "_id": {"producer_id": "$_id", "name": "$name", "year": "$movie_details.release_year"},
        "movie_count": {"$sum": 1}
    }},

        # {'_id': {'producer_id': 1, 'name': 'Karan', 'year': 2021}, 'movie_count': 1}
        # {'_id': {'producer_id': 3, 'name': 'Ekta', 'year': 2022}, 'movie_count': 2}
        # {'_id': {'producer_id': 4, 'name': 'Aditya', 'year': 2022}, 'movie_count': 2}
        # {'_id': {'producer_id': 5, 'name': 'Rohit', 'year': 2021}, 'movie_count': 1}
        # {'_id': {'producer_id': 5, 'name': 'Rohit', 'year': 2022}, 'movie_count': 1}
        # {'_id': {'producer_id': 2, 'name': 'Yash', 'year': 2021}, 'movie_count': 1}
        # {'_id': {'producer_id': 1, 'name': 'Karan', 'year': 2022}, 'movie_count': 1}


    {"$match": {"movie_count": {"$gt": 1}}},
        # {'_id': {'producer_id': 3, 'name': 'Ekta', 'year': 2022}, 'movie_count': 2}
        # {'_id': {'producer_id': 4, 'name': 'Aditya', 'year': 2022}, 'movie_count': 2}
    {"$project": {"_id": 0, "PID": "$_id.producer_id", "name": "$_id.name", "year": "$_id.year", "movie_count": 1}}
    

]

producers_with_multiple_movies = list(producers_collection.aggregate(pipeline))
for producer in producers_with_multiple_movies:
    print(producer)

{'movie_count': 2, 'PID': 4, 'name': 'Aditya', 'year': 2022}
{'movie_count': 2, 'PID': 3, 'name': 'Ekta', 'year': 2022}


In [22]:
client.close()
